<h1> Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto </h1>

<h2> Question 1 </h2>

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line even if you hyou are using geopy first time in IBM Watson studio
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup
from urllib.request import urlopen

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you are using folium first time in IBM Watson studio
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3> Getting the data </h3>

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

wiki_table = soup.body.table.tbody

In [14]:
def get_cell(element):
    cells = element.find_all('td')
    row = []
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
    return row
def get_row():    
    data = []  
    
    for tr in wiki_table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [15]:
data = get_row()
columns = ['PostalCode', 'Borough', 'Neighbourhood']
df_toronto = pd.DataFrame(data, columns=columns)
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [16]:
df_toronto.shape

(180, 3)

<h3> Dataset Cleaning </h3>

In [17]:
df1_toronto = df[df.Borough != 'Not assigned']
df1_toronto = df1.sort_values(by=['PostalCode','Borough'])

df1_toronto.reset_index(inplace=True)
df1_toronto.drop('index',axis=1,inplace=True)

df1_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
df_postcodes = df1_toronto['PostalCode']
df_postcodes.drop_duplicates(inplace=True)
df2_toronto = pd.DataFrame(df_postcodes)
df2_toronto['Borough'] = '';
df2_toronto['Neighbourhood'] = '';


df2_toronto.reset_index(inplace=True)
df2_toronto.drop('index', axis=1, inplace=True)
df1_toronto.reset_index(inplace=True)
df1_toronto.drop('index', axis=1, inplace=True)

for i in df2_toronto.index:
    for j in df1_toronto.index:
        if df2_toronto.iloc[i, 0] == df1_toronto.iloc[j, 0]:
            df2_toronto.iloc[i, 1] = df1_toronto.iloc[j, 1]
            df2_toronto.iloc[i, 2] = df2_toronto.iloc[i, 2] + ',' + df1_toronto.iloc[j, 2]
            
for i in df2_toronto.index:
    s = df2_toronto.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2_toronto.iloc[i,2 ] = s

In [11]:
df2_toronto.shape

(103, 3)

<h2> Question 2 </h2>

<h5> Using the Geocoder package or the csv file to create the following dataframe.
Adding the geospatial data to the data frame </h5> 

In [32]:
df2_toronto['Latitude'] = '0';
df2_toronto['Longitude'] = '0';

df_geo_toronto = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo_toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
for i in df2_toronto.index:
    for j in df_geo_toronto.index:
        if df2_toronto.iloc[i, 0] == df_geo_toronto.iloc[j, 0]:
            df2_toronto.iloc[i, 3] = df_geo_toronto.iloc[j, 1]
            df2_toronto.iloc[i, 4] = df_geo_toronto.iloc[j, 2]
            
df2_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


<h2> Question 3 </h2>
<h5>Clustering the Toronto neighborhoods</h5>

In [34]:
df3_toronto = df2_toronto.copy()
df3_toronto = df3_toronto[df2_toronto.Borough.str.contains("Toronto")]
df3_toronto.head()  

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6764,-79.293
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.669,-79.3156
43,M4M,East Toronto,Studio District,43.6595,-79.3409
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


In [35]:
toronto_clusters_neighborhoods = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = df3_toronto['Latitude']
Y = df3_toronto['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df3_toronto['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df3_toronto['Latitude'], df3_toronto['Longitude'], df3_toronto['Borough'], df3_toronto['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_clusters_neighborhoods)  

toronto_clusters_neighborhoods